In [1]:
import pandas as pd
from pyspark.sql import SparkSession
from pyspark.sql.dataframe import DataFrame as spark_DataFrame
from pyspark.sql.functions import desc

In [2]:
spark = SparkSession.builder.appName('LeetCode').getOrCreate()

25/01/03 00:46:32 WARN Utils: Your hostname, madiv resolves to a loopback address: 127.0.1.1; using 192.168.1.11 instead (on interface wlo1)
25/01/03 00:46:32 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/01/03 00:46:41 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [3]:
spark_cinema = spark.read.option('header', True).option('delimiter', ',').csv('Cinema.csv')
pandas_cinema = pd.read_csv('Cinema.csv')

spark_cinema.show(100)
pandas_cinema.head(100)

+---+----------+-----------+------+
| id|     movie|description|rating|
+---+----------+-----------+------+
|  1|       War|   great 3D|   8.9|
|  2|   Science|    fiction|   8.5|
|  3|     irish|     boring|   6.2|
|  4|  Ice Song|    Fantacy|   8.6|
|  5|House Card|Interesting|   9.1|
+---+----------+-----------+------+



,id,movie,description,rating
0,1,War,great 3D,8.9
1,2,Science,fiction,8.5
2,3,irish,boring,6.2
3,4,Ice Song,Fantacy,8.6
4,5,House Card,Interesting,9.1


### SQL

In [4]:
def not_boring_movies(cinema: spark_DataFrame) -> spark_DataFrame:
    # Create or replace a temporary SQL view named 'Cinema' from the input DataFrame
    cinema.createOrReplaceTempView('Cinema')

    # Define the SQL query to select movies that are not boring and have an odd id
    sqlQuery = \
    '''
    SELECT * FROM CINEMA
    WHERE description != 'boring' AND id % 2 = 1  -- Filter: description is not 'boring' and id is odd
    ORDER BY rating DESC  -- Order the results by rating in descending order
    '''
    
    # Execute the SQL query on the temporary view 'Cinema' and store the result in 'not_Boring_Movies'
    not_Boring_Movies = spark.sql(sqlQuery)  # Note: The 'sqlQuery' variable is passed directly to sql()

    # Return the filtered and ordered DataFrame
    return not_Boring_Movies

output = not_boring_movies(cinema = spark_cinema)
output.show(100)

+---+----------+-----------+------+
| id|     movie|description|rating|
+---+----------+-----------+------+
|  5|House Card|Interesting|   9.1|
|  1|       War|   great 3D|   8.9|
+---+----------+-----------+------+



### PySpark

In [5]:
def not_boring_movies(cinema: spark_DataFrame) -> spark_DataFrame:
    # Apply a filter to the 'cinema' DataFrame to select rows where:
    # 1. The 'description' column is not 'boring'
    # 2. The 'id' column is an odd number (id % 2 == 1)
    not_Boring_Movies = cinema.filter(
                                # Filter out 'boring' descriptions
                                (cinema['description'] != 'boring') &  
                                # Include only rows where 'id' is odd
                                (cinema['id'] % 2 == 1))\
                            .orderBy(
                                # Order the filtered rows by 'rating' 
                                # in descending order (i.e., higher ids first)
                                cinema.rating.desc())  

    # Return the filtered and ordered DataFrame containing non-boring movies with odd ids
    return not_Boring_Movies

output = not_boring_movies(cinema = spark_cinema)
output.show(100)

+---+----------+-----------+------+
| id|     movie|description|rating|
+---+----------+-----------+------+
|  5|House Card|Interesting|   9.1|
|  1|       War|   great 3D|   8.9|
+---+----------+-----------+------+



### Pandas

In [6]:
def not_boring_movies(cinema: pd.DataFrame) -> pd.DataFrame:
    # Filter the 'cinema' DataFrame to select rows where:
    # 1. The 'description' column is not equal to 'boring'
    # 2. The 'id' column is an odd number (id % 2 == 1)
    not_Boring_Movies = cinema[
        (cinema['description'] != 'boring') &   # Condition: exclude movies with 'boring' in description
        (cinema['id'] % 2 == 1)                 # Condition: include only rows where 'id' is odd
    ]
    
    # Sort the filtered DataFrame by the 'rating' column in descending order (highest rating first)
    # The 'inplace=True' argument modifies the DataFrame in place, so no need to assign it back.
    not_Boring_Movies.sort_values(by = ['rating'], ascending = False, inplace = True)
    
    # Return the filtered and sorted DataFrame containing non-boring movies with odd ids
    return not_Boring_Movies

output = not_boring_movies(cinema = pandas_cinema)
output.head(100)

/tmp/ipykernel_15507/1200089780.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  not_Boring_Movies.sort_values(by = ['rating'], ascending = False, inplace = True)


,id,movie,description,rating
4,5,House Card,Interesting,9.1
0,1,War,great 3D,8.9
